<a href="https://colab.research.google.com/github/sukmoonlee/JupyterNotebook/blob/main/2020/HP_Repository_Report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## HP Repository를 이용한 펌웨어 정보 추출하기

In [1]:
%%capture
!pip install pandas_read_xml

In [2]:
import pandas as pd
import pandas_read_xml as pdx
import datetime

## Repository에서 정보 추출하기

In [3]:
def GetRepoInfo(url):
  repo_df = pdx.read_xml(url + "/repodata/repomd.xml",  ['repomd', 'data'])
  primary = url + repo_df[repo_df['@type']=='primary']['location'].values[0]['@href']
  !wget "{primary}" -O primary.xml.gz &>/dev/null
  !gzip -df primary.xml.gz
  return pdx.read_xml("primary.xml",  ['metadata', 'package'])

In [4]:
# # test code

# work_df = GetRepoInfo("http://downloads.linux.hpe.com/SDR/repo/spp-g7/RHEL/7/x86_64/current/")
# print("Package Count : ", len(work_df))
# work_df.head(2)

In [5]:
def InsertArrayDF(df, model, name, arch, time, version, release, location):
  global idx_cnt
  
  df.loc[idx_cnt, 'Model']=model
  df.loc[idx_cnt, 'Name']=name
  df.loc[idx_cnt, 'Arch']=arch
  df.loc[idx_cnt, 'time']=datetime.datetime.fromtimestamp(int(time)).strftime('%Y-%m-%d')
  df.loc[idx_cnt, 'version']=version
  df.loc[idx_cnt, 'release']=release
  df.loc[idx_cnt, 'location']=location
  idx_cnt += 1

def ExtractSmartArray(work_df):
  excel = pd.DataFrame()

  for idx in work_df.index:
    name=work_df.loc[idx, 'name']
    if name.find("firmware-smartarray")<0: continue
    
    arch=work_df.loc[idx, 'arch']
    tlist=work_df.loc[idx, 'time'].values()
    time=list(tlist)[1]
    tlist=work_df.loc[idx, 'version'].values()
    version=list(tlist)[1]
    release=list(tlist)[2]
    summary=work_df.loc[idx, 'summary']
    tlist=work_df.loc[idx, 'location'].values()
    location=list(tlist)[0]
    #print(name, arch, time, version, release, location, summary)

    str = summary
    str = str.replace(", and", ",")
    str = str.replace(" and", ",")
    #print(str)

    if str[0:11]=="Smart Array":
      str = str.replace("Smart Array", "")
      # print(str)

    if str.find(",")<0:
      model = str.replace("firmware", "")
      #print(model, name, arch, time, version, release, location)
      InsertArrayDF(excel, model, name, arch, time, version, release, location)
    
    else:
      tlist = str.split(sep=",")    
      for i in tlist:
        model1 = i.strip()
        model2 = model1.split(sep=" ")
        model = model2[0]      
        #print(model, name, arch, time, version, release, location)
        InsertArrayDF(excel, model, name, arch, time, version, release, location)

  return excel

def ExtractHDD(work_df):
  excel = pd.DataFrame()

  for idx in work_df.index:
    name=work_df.loc[idx, 'name']
    if name.find("firmware-hdd")<0: continue
    
    arch=work_df.loc[idx, 'arch']
    tlist=work_df.loc[idx, 'time'].values()
    time=list(tlist)[1]
    tlist=work_df.loc[idx, 'version'].values()
    version=list(tlist)[1]
    release=list(tlist)[2]
    summary=work_df.loc[idx, 'summary']
    tlist=work_df.loc[idx, 'location'].values()
    location=list(tlist)[0]
    #print(name, arch, time, version, release, location, summary)

    str = summary
    str = str.replace(", and", ",")
    str = str.replace(" and", ",")
    #print(str)

    if str.find(",")<0:
      model1 = str.strip()
      model2 = model1.split(sep=" ")
      model = model2[0]
      #print(model, name, arch, time, version, release, location)
      InsertArrayDF(excel, model, name, arch, time, version, release, location)
    
    else:
      tlist = str.split(sep=",")    
      for i in tlist:
        model1 = i.strip()
        model2 = model1.split(sep=" ")
        model = model2[0]
        #print(model, name, arch, time, version, release, location)
        InsertArrayDF(excel, model, name, arch, time, version, release, location)

  return excel

In [6]:
# # test code

# idx_cnt = 0
# rhel7_gen7_df = GetRepoInfo("http://downloads.linux.hpe.com/SDR/repo/spp-g7/RHEL/7/x86_64/current/")
# print("Package Count : ", len(rhel7_gen7_df))
# result_gen7_df = ExtractSmartArray(rhel7_gen7_df, "http://downloads.linux.hpe.com/SDR/repo/spp-g7/RHEL/7/x86_64/current/")
# result_gen7_df['Gen']="G7"
# print("Report Package Count : ", len(result_gen7_df))
# result_gen7_df.head(2)

In [7]:
# # test code - HDD

# idx_cnt = 0
# df = GetRepoInfo("http://downloads.linux.hpe.com/SDR/repo/spp-g7/RHEL/7/x86_64/current/")
# print("Package Count : ", len(df))
# result_df = ExtractHDD(df)
# print("Report Package Count : ", len(result_df))
# result_df

## 결과 리포트하기

In [8]:
url_list = {
    "Gen7" : "http://downloads.linux.hpe.com/SDR/repo/spp-g7/RHEL/7/x86_64/current/",
    "Gen8" : "http://downloads.linux.hpe.com/SDR/repo/spp-gen8/RHEL/7/x86_64/current/",
    "Gen9" : "http://downloads.linux.hpe.com/SDR/repo/spp-gen9/RHEL/7/x86_64/current/",
    "Gen10" : "http://downloads.linux.hpe.com/SDR/repo/spp-gen10/RHEL/7/x86_64/current/",
}

report_array_df = pd.DataFrame()
report_hdd_df = pd.DataFrame()

idx_cnt = 0
for url_idx in url_list:
  #print(url_idx, url_list[url_idx])
  df = GetRepoInfo(url_list[url_idx])
  print(url_idx, len(df))
  result_df = ExtractSmartArray(df)
  result_df['Gen']=url_idx
  result_df['location']=url_list[url_idx] + result_df['location']
  report_array_df = pd.concat([report_array_df, result_df], ignore_index=True, join='outer')

  result_df = ExtractHDD(df)
  result_df['Gen']=url_idx
  report_hdd_df = pd.concat([report_hdd_df, result_df], ignore_index=True, join='outer')

Gen7 722
Gen8 730
Gen9 2222
Gen10 2222


In [9]:
report_array_df.head(2)
report_hdd_df.head(2)

,Model,Name,Arch,time,version,release,location,Gen
0,MB1000FAMYU,hp-firmware-hdd-2db44cb024,x86_64,2015-01-13,HPD7,2,hp-firmware-hdd-2db44cb024-HPD7-2.x86_64.rpm,Gen7
1,MB2000FAMYV,hp-firmware-hdd-2db44cb024,x86_64,2015-01-13,HPD7,2,hp-firmware-hdd-2db44cb024-HPD7-2.x86_64.rpm,Gen7


In [10]:
# 특정 Array Controller 모델 확인하기
report_array_df[ (report_array_df['Model']=='P212') & (report_array_df['Gen']=='Gen7') ].sort_values(by='time', ascending=False)

,Model,Name,Arch,time,version,release,location,Gen
87,P212,hp-firmware-smartarray-14ef73e580,i386,2015-07-10,6.64,2,http://downloads.linux.hpe.com/SDR/repo/spp-g7...,Gen7
94,P212,hp-firmware-smartarray-14ef73e580,x86_64,2015-07-10,6.64,2,http://downloads.linux.hpe.com/SDR/repo/spp-g7...,Gen7
40,P212,hp-firmware-smartarray-14ef73e580,i386,2015-03-27,6.64,1.1,http://downloads.linux.hpe.com/SDR/repo/spp-g7...,Gen7
47,P212,hp-firmware-smartarray-14ef73e580,x86_64,2015-03-27,6.64,1.1,http://downloads.linux.hpe.com/SDR/repo/spp-g7...,Gen7
6,P212,hp-firmware-smartarray-14ef73e580,x86_64,2015-01-14,6.62,1.1,http://downloads.linux.hpe.com/SDR/repo/spp-g7...,Gen7


In [11]:
# 특정 HDD 모델 확인하기
report_hdd_df[ (report_hdd_df['Model']=='MB3000EBKAB') & (report_hdd_df['Gen']=='Gen7') ].sort_values(by='time', ascending=False)

,Model,Name,Arch,time,version,release,location,Gen
257,MB3000EBKAB,hp-firmware-hdd-3675aa63c9,x86_64,2016-02-11,HPG6,4.1,hp-firmware-hdd-3675aa63c9-HPG6-4.1.x86_64.rpm,Gen7
117,MB3000EBKAB,hp-firmware-hdd-3675aa63c9,x86_64,2015-04-16,HPG6,3,hp-firmware-hdd-3675aa63c9-HPG6-3.x86_64.rpm,Gen7
2,MB3000EBKAB,hp-firmware-hdd-3675aa63c9,x86_64,2014-11-06,HPG6,2,hp-firmware-hdd-3675aa63c9-HPG6-2.x86_64.rpm,Gen7


In [12]:
# 특정 펌웨어 URL 확인하기
print(report_array_df.loc[87, 'location'])

http://downloads.linux.hpe.com/SDR/repo/spp-g7/RHEL/7/x86_64/current/hp-firmware-smartarray-14ef73e580-6.64-2.i386.rpm


In [13]:
# 엑세로 저장하기
report_array_df.to_excel("HP-Firmware-SmartArray.xls")
report_hdd_df.to_excel("HP-Firmware-HDD.xls")